In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
cmap = matplotlib.cm.get_cmap('Spectral')

import numpy as np

from sklearn.datasets import make_moons, make_hastie_10_2
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier

In [ ]:
## Funciones útiles

def print_results(clfrgr):
    print('#####')
    print('{}.'.format(type(clfrgr).__name__))
                       
    train_score = clfrgr.score(X_train, y_train)
    test_score = clfrgr.score(X_test, y_test)
    print('Train score: ', train_score)
    print('Test score: ', test_score)

def print_results_boost(clfrgr):
    print('#####')
    print('{} con {} estimadores de tipo {}, con max_depth={}.'.format(type(clfrgr).__name__,
                                                                       clfrgr.n_estimators, 
                                                                       type(clfrgr.base_estimator).__name__,
                                                                       clfrgr.base_estimator.max_depth))
    train_score = clfrgr.score(X_train, y_train)
    test_score = clfrgr.score(X_test, y_test)
    print('Train score: ', train_score)
    print('Test score: ', test_score)

    return train_score, test_score

def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.5, -1, 1.5], alpha=0.5, contour=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    ax = plt.gca()
    
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "o", color=cmap(0), alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "yo", alpha=alpha)
    plt.axis(axes)
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

## Bagging

In [ ]:
X, y = make_moons(5000, shuffle=True, noise=0.2)
size_train = int(0.8 * len(X))
X_train, X_test = X[:size_train], X[size_train:]
y_train, y_test = y[:size_train], y[size_train:]

plt.scatter(*X_train.T, c=y_train, marker='.')
#plt.scatter(*X_test.T, c=y_test, marker='^')

In [ ]:
dt = DecisionTreeClassifier(max_depth=12, random_state=107)
dt = dt.fit(X_train, y_train)

# svm_pipe = Pipeline(steps=[('scaler', StandardScaler()), 
#                            ('svc', SVC(kernel='poly'))])
# svm_pipe = svm_pipe.fit(X_train, y_train)

plot_decision_boundary(dt, X_train, y_train, contour=True, alpha=0.1)
print_results(dt)

In [ ]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(DecisionTreeClassifier(max_depth=12, random_state=107), n_estimators=500, n_jobs=-1, 
                        oob_score=False, max_samples=1000, bootstrap=True)
clf = clf.fit(X_train, y_train)

In [ ]:
print(clf.oob_decision_function_.shape)
print(clf.oob_score_)
print(clf.oob_decision_function_[:5])

In [ ]:
plot_decision_boundary(clf, X_train, y_train, alpha=0.1)
print_results(clf)



### Extra trees

In [ ]:
from sklearn.tree import ExtraTreeClassifier

dt = ExtraTreeClassifier(max_depth=12, random_state=107)
dt = dt.fit(X_train, y_train)

# svm_pipe = Pipeline(steps=[('scaler', StandardScaler()), 
#                            ('svc', SVC(kernel='poly'))])
# svm_pipe = svm_pipe.fit(X_train, y_train)

plot_decision_boundary(dt, X_train, y_train, contour=True, alpha=0.1)
print_results(dt)

In [ ]:
clf = BaggingClassifier(ExtraTreeClassifier(max_depth=12, random_state=107), n_estimators=500, n_jobs=-1, 
                        oob_score=False, max_samples=1000, bootstrap=True)
clf = clf.fit(X_train, y_train)

plot_decision_boundary(clf, X_train, y_train, alpha=0.1)
print_results(clf)

### Bias-Variance

In [ ]:
dt = DecisionTreeClassifier(max_depth=1, random_state=107)
dt = dt.fit(X_train, y_train)

# svm_pipe = Pipeline(steps=[('scaler', StandardScaler()), 
#                            ('svc', SVC(kernel='poly'))])
# svm_pipe = svm_pipe.fit(X_train, y_train)

plot_decision_boundary(dt, X_train, y_train, contour=True, alpha=0.1)
print_results(dt)

In [ ]:
clf = BaggingClassifier(DecisionTreeClassifier(max_depth=1, random_state=107), n_estimators=500, n_jobs=-1, 
                        oob_score=False, max_samples=1000, bootstrap=True)
clf = clf.fit(X_train, y_train)

plot_decision_boundary(clf, X_train, y_train, alpha=0.1)
print_results(clf)

### Boosting

Para mostrar el poder de boosting, hagamos un nuevo dataset.

In [ ]:
X, y = make_hastie_10_2(random_state=0)

print(X.shape)

X_train, X_test = X[2000:], X[:2000]
y_train, y_test = y[2000:], y[:2000]

Este dataset consiste en diez features normales estándares, con los siguientes labels: 

`y[i] = 1 if np.sum(X[i] ** 2) > 9.34 else -1`

Es decir, es una hiperesfera adentro de una hiperesfera hueca.

In [ ]:
import pandas as pd
import seaborn as sns

dd = pd.DataFrame.from_records(X_train)
dd['target'] = y_train

#ss.pairplot(dd.iloc[::20, :5], hue='target')
#ss.pairplot(dd.iloc[::20], hue='target', vars=range(5), markers=',')
sns.pairplot(dd.iloc[::20], hue='target', vars=range(4), markers='.')


Veamos como funciona `AdaBoost` en un caso así. Combinemos DTs con profundidad máxima 2.

In [ ]:
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=1.0)

clf.fit(X_train, y_train)
print_results_boost(clf)

In [ ]:
for nesti in [10, 20, 50, 75]:
    clf.set_params(n_estimators=nesti)
    clf.fit(X_train, y_train)
    print_results_boost(clf)

clf.set_params(n_estimators=100)
clf.base_estimator.set_params(max_depth=1)
clf.fit(X_train, y_train)
print_results(clf)

### Stacking

In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.svm import SVR
estimators = [('ridge', RidgeCV()),
              ('lasso', LassoCV(random_state=42)),
              ('svr', SVR(C=1, gamma=1e-6))]

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor

reg = StackingRegressor(estimators=estimators, final_estimator=GradientBoostingRegressor(random_state=42))

In [ ]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Y se ajusta como siempre, con la magia de sklearn
reg.fit(X_train, y_train)

In [ ]:
reg.score(X_test, y_test)

Y podemos obtener el output de cada uno con el método `transform`.

In [ ]:
reg.transform(X_test)